In [1]:
from settings import config, BASE_DIR

In [2]:
# get environment variables

OPENAI_API_KEY = config('OPENAI_API_KEY', default=None)
QDRANT_API_KEY = config('QDRANT_API_KEY', default=None)
QDRANT_CLUSTER_URL = config('QDRANT_CLUSTER_URL', default=None)

OPENAI_API_KEY is not None and QDRANT_API_KEY is not None and QDRANT_CLUSTER_URL is not None

True

In [3]:
doc = BASE_DIR / "data" / "story.txt"
doc.exists()

True

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
import qdrant_client
from qdrant_client.http import models

In [5]:
# create a qdrant client

qdrant = qdrant_client.QdrantClient(
    url=QDRANT_CLUSTER_URL, 
    api_key=QDRANT_API_KEY
)

In [7]:
# create a collection

vector_config = models.VectorParams(
    size=1536, # for openai embeddings 
    distance=models.Distance.COSINE
)

qdrant.create_collection(
    collection_name="my_collection",
    vectors_config=vector_config,
)

True

In [11]:
# get our collection info

qdrant.get_collection(collection_name="my_collection")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [12]:
# create a vector store
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vector_store = Qdrant(
    client=qdrant,
    collection_name="my_collection",
    embeddings=embeddings,
)

In [13]:
# add documents to the vector store

# a function to split the text into chunks
def split_text(text: str, chunk_size: int = 1000, chunk_overlap: int = 100) -> list[str]:
    my_text_splitter = CharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separator="\n",
        length_function=len,
    )
    
    chunks = my_text_splitter.split_text(text)
    return chunks

# read the text
with open(doc, "r") as f:
    raw_text = f.read()

# split the text into chunks
text_chunks = split_text(raw_text)
print(text_chunks)

['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.']


In [14]:
# add chunks to the vector store

vector_store.add_texts(text_chunks)

['c73f2a41920542b5b52e439cd1b9677a']

In [15]:
# get our collection info test if the vectors are added

qdrant.get_collection(collection_name="my_collection")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=1, indexed_vectors_count=0, points_count=1, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [16]:
# use our vector store into a QA retrieval chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

QA = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY),
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

In [19]:
# ask a question

query = "What did the president say about Ketanji Brown Jackson"
res = QA.run(query)
print(res)

 The president nominated Ketanji Brown Jackson to serve on the United States Supreme Court.
